In [34]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
%store -r reverselookup


model = load_model('getsure_model.keras')  

roi_top, roi_bottom, roi_right, roi_left = 100, 400, 300, 600

arr=['palm','I','fist','fist_moved','thumb','index','ok','palm_moved','c','down']


cap = cv2.VideoCapture(0)
cap.set(3, 500)  # Width
cap.set(4, 500)
print("Frame Width:", cap.get(3))
print("Frame Height:", cap.get(4))


Frame Width: 640.0
Frame Height: 480.0


In [36]:
while True:
    ret, frame = cap.read()

    # Flip the frame horizontally for a later selfie-view display
    # frame = cv2.flip(frame, 1)

    
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # Preprocess the ROI for the model
    img = cv2.resize(roi, (320, 120))  # Resize to match the model input size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    img = img / 255.0  # Normalize pixel values to be between 0 and 1

    # Make a prediction with the model
    prediction = model.predict(img)

    # Get the predicted class
    gesture_class_index = np.argmax(prediction)

    gesture_class=reverselookup[gesture_class_index]

    # Display the gesture class on the frame
    cv2.putText(frame, f"Gesture: {gesture_class}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Draw the ROI on the frame
    cv2.rectangle(frame, (roi_left, roi_top), (roi_right, roi_bottom), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Gesture Recognition', frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 56ms/step
